In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import beta, norm, gamma
import ipywidgets as widgets
from IPython.display import display, clear_output

# Set the style
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.Dark2.colors)
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['text.color'] = 'black'
plt.rcParams['axes.labelcolor'] = 'black'
plt.rcParams['xtick.color'] = 'black'
plt.rcParams['ytick.color'] = 'black'
plt.rcParams['font.family'] = 'monospace'
plt.rcParams['font.size'] = 8
plt.rcParams['font.weight'] = 'normal'

def format_number(num):
    if num >= 0.001 or num <= -0.001:
        return "{:.4f}".format(num)
    else:
        return "{:.1e}".format(num)
    
# General function for plotting conjugacy
def plot_conjugacy(prior_pdf_values, posterior_pdf_values, ci_lower, ci_upper, title, details, x_values, mle, map_estimate, histogram_data=None):
    fig, ax1 = plt.subplots(figsize=(10, 4), dpi=400)

    ax1.plot(x_values, prior_pdf_values, label='Prior', color=plt.cm.Dark2.colors[2], zorder=5)
    ax1.plot(x_values, posterior_pdf_values, label='Posterior', color=plt.cm.Dark2.colors[0], zorder=5)

    ax1.axvline(ci_lower, color='grey', linestyle='--', label='95% C.I.', linewidth=1, zorder=5)
    ax1.axvline(ci_upper, color='grey', linestyle='--', linewidth=1, zorder=5)

    # Plot MLE and MAP
    if mle is not None:
        ax1.axvline(mle, color='orange', label=f'MLE={format_number(mle)}', linestyle='-.', linewidth=1, zorder=5)
    if map_estimate is not None:
        ax1.axvline(map_estimate, color=plt.cm.Dark2.colors[3], label=f'MAP={format_number(map_estimate)}', linestyle='-.', linewidth=1, zorder=5)
    
    ax1.set_ylim(0, max(np.max(prior_pdf_values), np.max(posterior_pdf_values)) * 1.1)
    ax1.set_xlabel('θ')
    ax1.set_ylabel('Density')
    ax1.set_title(title)

    if histogram_data is not None:
        # Filter histogram data to be within the range of x_values
        filtered_data = histogram_data[(histogram_data >= x_values[0]) & (histogram_data <= x_values[-1])]

        # Calculate number of bins dynamically
        num_bins = min(len(filtered_data), 50)
        ax2 = ax1.twinx()
        ax2.hist(filtered_data, bins=num_bins, alpha=0.3, color='grey', rwidth=0.9, zorder=1)
        ax2.set_ylabel('Sample Count', color='black')
        ax2.tick_params(axis='y', labelcolor='black')
        ax2.yaxis.set_major_locator(plt.MaxNLocator(integer=True))

    if title == 'Beta-Binomial Conjugacy':
        ax1.set_xlim(0, 1)
        ax1.set_xlabel('θ (probability of success)')
        
    if title == 'Gaussian-Gaussian Conjugacy':
        ax1.set_xlim(x_values[0], x_values[-1])
        ax1.set_xlabel('μ (mean)')
    
    if title == 'Gamma-Poisson Conjugacy':
        ax1.set_xlim(x_values[0], x_values[-1])
        ax1.set_xlabel('λ (rate)')
        
    if title == 'Beta-Negative Binomial Conjugacy':
        ax1.set_xlim(0, 1)
        ax1.set_xlabel('θ (probability of success)')
    
    ax1.legend()
    plt.show()

    for detail in details:
        print("    " + detail)



# Function for Beta-Binomial conjugacy
def plot_beta_binomial(alpha_prior, beta_prior, trails_likelihood, p_success_likelihood):
    # Binomial flip
    successes = np.random.binomial(trails_likelihood, p_success_likelihood)
    failures = trails_likelihood - successes

    # Update alpha_prior and beta for posterior
    alpha_post = alpha_prior + successes
    theta_post = beta_prior + failures

    # PDF values
    x_values = np.linspace(0, 1, 1000)
    prior_pdf_values = beta.pdf(x_values, alpha_prior, beta_prior)
    posterior_pdf_values = beta.pdf(x_values, alpha_post, theta_post)

    # Calculating 95% Confidence Interval
    ci_lower = beta.ppf(0.025, alpha_post, theta_post)
    ci_upper = beta.ppf(0.975, alpha_post, theta_post)

    # Calculate MLE and MAP
    mle = successes / trails_likelihood
    map_estimate = (alpha_prior + successes - 1) / (alpha_prior + beta_prior + trails_likelihood - 2)

    # Details
    title = 'Beta-Binomial Conjugacy'
    details = [
        f"{trails_likelihood} trails with {successes} successes ({successes / trails_likelihood * 100:.2f}%) and {failures} failures ({failures / trails_likelihood * 100:.2f}%)",
        f"Prior: h(θ) ~ Beta(α={alpha_prior:.2f}, β={beta_prior:.2f})",
        f"Likelihood: g(x|θ) ~ Binomial(n={trails_likelihood}, p={p_success_likelihood:.2f})",
        f"Posterior: f(θ|x) ~ Beta(α={alpha_post:.2f}, β={theta_post:.2f})",
    ]
    
    plot_conjugacy(prior_pdf_values, posterior_pdf_values, ci_lower, ci_upper, title, details, x_values, mle, map_estimate)

def plot_gaussian_gaussian(mu_prior, sigma_prior, sigma_likelihood, n_samples):
    # Generate samples
    samples = np.random.normal(mu_prior, sigma_likelihood, n_samples)

    # Posterior parameters
    sigma_post = np.sqrt(1.0 / (1.0 / sigma_prior**2 + n_samples / sigma_likelihood**2))
    mu_post = sigma_post**2 * (mu_prior / sigma_prior**2 + np.sum(samples) / sigma_likelihood**2)

    # MLE and MAP
    mle = np.mean(samples)
    map_estimate = mu_post  # For Gaussian, MAP is the mean of the posterior

    # 95% CI for posterior
    ci_lower, ci_upper = norm.interval(0.95, loc=mu_post, scale=sigma_post)

    # Adjust the range based on sample data
    x_min = mu_prior - 3 * sigma_prior
    x_max = mu_prior + 3 * sigma_prior
    x_values = np.linspace(x_min, x_max, 1000)

    # PDF values
    prior_pdf_values = norm.pdf(x_values, mu_prior, sigma_prior)
    posterior_pdf_values = norm.pdf(x_values, mu_post, sigma_post)

    # Details
    title = 'Gaussian-Gaussian Conjugacy'
    details = [
        f"{n_samples} samples with mean={mle:.2f} and std={np.std(samples):.2f}",
        f"Prior: h(μ) ~ N(μ={mu_prior:.2f}, σ²={sigma_prior**2:.2f})",
        f"Likelihood: g(x|μ) ~ N(μ, σ²={sigma_likelihood**2:.2f})",
        f"Posterior: f(μ|x) ~ N(μ={mu_post:.2f}, σ²={sigma_post**2:.2f})"
    ]
    
    plot_conjugacy(prior_pdf_values, posterior_pdf_values, ci_lower, ci_upper, title, details, x_values, mle, map_estimate, histogram_data=samples)

def plot_poisson_gamma(alpha_prior, theta_prior, n_samples):
    # Generate samples
    samples = np.random.poisson(theta_prior, n_samples)

    # Posterior parameters
    alpha_post = alpha_prior + np.sum(samples)
    theta_post = theta_prior + n_samples

    # MLE and MAP
    mle = np.mean(samples)
    map_estimate = (alpha_post - 1) / theta_post if alpha_post > 1 else 0

    # PDF values for Gamma distribution
    x_values = np.linspace(0, max(10, 1.5 * np.max(samples)), 1000)
    prior_pdf_values = gamma.pdf(x_values, alpha_prior, scale=1/theta_prior)
    posterior_pdf_values = gamma.pdf(x_values, alpha_post, scale=1/theta_post)

    ci_lower = gamma.ppf(0.025, alpha_post, scale=1/theta_post)
    ci_upper = gamma.ppf(0.975, alpha_post, scale=1/theta_post)

    # Details
    title = 'Gamma-Poisson Conjugacy'
    details = [
        f"{n_samples} samples with mean={mle:.2f} and std={np.std(samples):.2f}",
        f"Prior: h(λ) ~ Gamma(α={alpha_prior:.2f}, β={theta_prior:.2f})",
        f"Likelihood: g(x|λ) ~ Poisson(λ={theta_prior:.2f})",
        f"Posterior: f(λ|x) ~ Gamma(α={alpha_post:.2f}, β={theta_post:.2f})"
    ]
    
    plot_conjugacy(prior_pdf_values, posterior_pdf_values, ci_lower, ci_upper, title, details, x_values, mle, map_estimate, histogram_data=samples)

def plot_negative_binomial_beta(alpha_prior, beta_prior, num_successes, p_success_likelihood):
    # Generate negative binomial samples
    num_failures = np.random.negative_binomial(num_successes, p_success_likelihood)

    # Update alpha and beta for posterior
    alpha_post = alpha_prior + num_successes
    beta_post = beta_prior + num_failures

    # MAP estimate
    map_estimate = (alpha_post - 1) / (alpha_post + beta_post - 2) if alpha_post > 1 and beta_post > 1 else 0

    # Credible interval for the posterior
    ci_lower = beta.ppf(0.025, alpha_post, beta_post)
    ci_upper = beta.ppf(0.975, alpha_post, beta_post)

    # PDF values
    x_values = np.linspace(0, 1, 1000)
    prior_pdf_values = beta.pdf(x_values, alpha_prior, beta_prior)
    posterior_pdf_values = beta.pdf(x_values, alpha_post, beta_post)

    # Details
    title = 'Beta-Negative Binomial Conjugacy'
    details = [
        f"Observed {num_failures} failures before {num_successes} successes",
        f"Prior: h(θ) ~ Beta(α={alpha_prior:.2f}, β={beta_prior:.2f})",
        f"Likelihood: g(x|θ) ~ Negative Binomial(successes={num_successes}, p={p_success_likelihood:.2f})",
        f"Posterior: f(θ|x) ~ Beta(α={alpha_post:.2f}, β={beta_post:.2f})"
    ]
    
    plot_conjugacy(prior_pdf_values, posterior_pdf_values, ci_lower, ci_upper, title, details, x_values, None, map_estimate)

    
# Dropdown for selecting conjugacies
conjugacy_dropdown = widgets.Dropdown(
    options=["Select a Bayesian conjugacy", "Beta-Binomial", "Gaussian-Gaussian", "Gamma-Poisson", "Beta-Negative Binomial"],
    value="Select a Bayesian conjugacy",
    description='Conjugacy:'
)

# Output area
output_container = widgets.VBox([])  # Container to hold sliders and plots
custom_slider_layout = widgets.Layout(width='calc(100% - 110px)')

# Function to display corresponding widgets and plot
def display_distribution_widgets(change):
    if change['new'] == "Beta-Binomial":
        # Slider declarations with custom layouts
        alpha_slider = widgets.FloatSlider(value=1, min=0.01, max=10, step=0.01, description='α (prior):', continuous_update=False, layout=custom_slider_layout, style={'description_width': '110px'})
        beta_slider = widgets.FloatSlider(value=1, min=0.01, max=10, step=0.01, description='β (prior):', continuous_update=False, layout=custom_slider_layout, style={'description_width': '110px'})
        p_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.01, description='θ (p) (likelihood)', continuous_update=False, layout=custom_slider_layout, style={'description_width': '110px'})
        flips_slider = widgets.IntSlider(value=10, min=1, max=500, step=1, description='num. of samples:', continuous_update=False, layout=custom_slider_layout, style={'description_width': '110px'})

        pior_box = widgets.HBox([alpha_slider, beta_slider])
        likelihood_box = widgets.HBox([p_slider, flips_slider])
        beta_interactive = widgets.interactive(plot_beta_binomial, alpha_prior=alpha_slider, beta_prior=beta_slider, trails_likelihood=flips_slider, p_success_likelihood=p_slider)
        output_container.children = [pior_box, likelihood_box, beta_interactive.children[-1]]

    elif change['new'] == "Gaussian-Gaussian":
        # Slider declarations for Gaussian-Gaussian
        mu_slider = widgets.FloatSlider(value=0, min=-10, max=10, step=0.01, description='μ₀ (prior):', continuous_update=False, layout=custom_slider_layout, style={'description_width': '110px'})
        sigma_prior_slider = widgets.FloatSlider(value=1, min=0.1, max=5, step=0.01, description='σ₀ (prior):', continuous_update=False, layout=custom_slider_layout, style={'description_width': '110px'})
        sigma_likelihood_slider = widgets.FloatSlider(value=1, min=0.1, max=5, step=0.01, description='σ (likelihood):', continuous_update=False, layout=custom_slider_layout, style={'description_width': '110px'})
        n_samples_slider = widgets.IntSlider(value=30, min=5, max=500, step=1, description='num. of samples:', continuous_update=False, layout=custom_slider_layout, style={'description_width': '110px'})

        prior_box = widgets.HBox([mu_slider, sigma_prior_slider])
        likelihood_box = widgets.HBox([sigma_likelihood_slider, n_samples_slider])
        gaussian_interactive = widgets.interactive(plot_gaussian_gaussian, mu_prior=mu_slider, sigma_prior=sigma_prior_slider, sigma_likelihood=sigma_likelihood_slider, n_samples=n_samples_slider)
        output_container.children = [prior_box, likelihood_box, gaussian_interactive.children[-1]]
        
    elif change['new'] == "Gamma-Poisson":
        # Slider declarations for Poisson-Gamma
        alpha_slider = widgets.FloatSlider(value=2, min=0.1, max=10, step=0.1, description='α (prior):', continuous_update=False, layout=custom_slider_layout, style={'description_width': '110px'})
        theta_slider = widgets.FloatSlider(value=2, min=0.1, max=10, step=0.1, description='θ (prior):', continuous_update=False, layout=custom_slider_layout, style={'description_width': '110px'})
        n_samples_slider_pg = widgets.IntSlider(value=30, min=5, max=100, step=1, description='num. of samples:', continuous_update=False, layout=custom_slider_layout, style={'description_width': '110px'})

        box_pg = widgets.HBox([alpha_slider, theta_slider, n_samples_slider_pg])
        poisson_interactive = widgets.interactive(plot_poisson_gamma, alpha_prior=alpha_slider, theta_prior=theta_slider, n_samples=n_samples_slider_pg)
        output_container.children = [box_pg, poisson_interactive.children[-1]]
    
    elif change['new'] == "Beta-Negative Binomial":
        alpha_slider_nb = widgets.FloatSlider(value=2, min=0.1, max=10, step=0.1, description='α (Beta prior):', continuous_update=False, layout=custom_slider_layout, style={'description_width': '110px'})
        beta_slider_nb = widgets.FloatSlider(value=2, min=0.1, max=10, step=0.1, description='β (Beta prior):', continuous_update=False, layout=custom_slider_layout, style={'description_width': '110px'})
        successes_slider = widgets.IntSlider(value=5, min=1, max=20, step=1, description='θ (p) (likelihood)', continuous_update=False, layout=custom_slider_layout, style={'description_width': '110px'})
        p_slider_nb = widgets.FloatSlider(value=0.5, min=0.01, max=1, step=0.01, description='r (likelihood):', continuous_update=False, layout=custom_slider_layout, style={'description_width': '110px'})

        prior_box_nb = widgets.HBox([alpha_slider_nb, beta_slider_nb])
        likelihood_box_nb = widgets.HBox([successes_slider, p_slider_nb])
        negative_binomial_interactive = widgets.interactive(plot_negative_binomial_beta, alpha_prior=alpha_slider_nb, beta_prior=beta_slider_nb, num_successes=successes_slider, p_success_likelihood=p_slider_nb)
        output_container.children = [prior_box_nb, likelihood_box_nb, negative_binomial_interactive.children[-1]]

        
# Observer
conjugacy_dropdown.observe(display_distribution_widgets, names='value')

# Initial display setup
display(conjugacy_dropdown, output_container)

Dropdown(description='Conjugacy:', options=('Select a Bayesian conjugacy', 'Beta-Binomial', 'Gaussian-Gaussian…

VBox()